In [1]:
from ltp import LTP
import torch
from transformers import AutoTokenizer, AutoModel

In [2]:
# 文句相似度 測試

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModel.from_pretrained("bert-base-chinese")

def get_bert_embedding(text):
    input_ids = torch.tensor([tokenizer.encode(text)])
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]
    return last_hidden_states[0][0]

def cosine_similarity(a, b):
    return torch.dot(a, b) / (torch.norm(a) * torch.norm(b))

def compare(text1,text2):

    embedding1 = get_bert_embedding(text1)
    embedding2 = get_bert_embedding(text2)

    similarity = cosine_similarity(embedding1, embedding2)

    return similarity



Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
class Get_Name:

    def __init__(self,model="LTP/base1",GPU=True):
        self.ltp = LTP("LTP/base1")
        #ltp = LTP(path = "LTP/base|LTP/small|LTP/tiny")

        # GPU 
        if GPU and torch.cuda.is_available():
            self.ltp.to("cuda")
    
    def get_name(self,text):
        


        slice_texts = text.split("\n")
        # cws 分詞 : 將文句拆開
        # pos 詞性標註 ， 將拆開的文句用詞性分類 ，名稱 : nh
        # ner 命名實體
        # 與文句的用詞也有關係
        names = []

        for slice_text in slice_texts:
            words = self.ltp.pipeline([slice_text], tasks = ["cws","pos","ner","srl","sdp"])
            for name in words["ner"][0]:

                if name[0] == "Nh" and name[1] not in names:
                    names.append(name[1])
            maybe_names = self.find_noun(words["srl"])
            
            for maybe_name in maybe_names:
                if maybe_names[maybe_name] >= 2 :
                    if self.check_noun(maybe_name) and maybe_name not in names:
                        names.append(maybe_name)
        # copy_names = names.copy()
        # for i in range(len(copy_names)):
        #     for j in range(i+1,len(copy_names)):
        #         compare(copy_names[i],copy_names[j])


        return names


    def find_noun(self,words):
        
        maybe_names = {}
        for i in words[0]:
            for j in i["arguments"]:
                if j[0] == "A0":
                    try:
                        maybe_names[j[1]]+=1
                    except:
                        maybe_names.setdefault(j[1],1)

        return maybe_names

    def check_noun(self,text):
        
        words = self.ltp.pipeline([text], tasks = ["pos"])
        # print(text)
        check_table = ["nh","n"]
        for part_of_speech in words["pos"]:
            # print(part_of_speech)
            if part_of_speech not in check_table:
                return False
        
        return True

In [12]:
# text = """
# 陸無雙叫道：「表姊，快逃。」拉著程英的手轉身便走。程英奔出幾步，只見怪客頭上汩汩冒血，心中不忍，道：「老伯伯別撞死啦，瞧瞧他去。」
# 陸無雙道：「死了，那不變了鬼麼？」程英吃了一驚，既怕他變鬼，又怕他忽然醒轉，再抓住自己說些古裏古怪的瘋話，可是見他滿臉鮮血，甚為可憐，
# 自己安慰自己：「老伯伯不是鬼，我不怕，他不會再抓我。」一步步的緩緩走近，叫道：「老伯伯，你痛麼？」
# """
f = open('./story.txt')
text = f.read()
GN = Get_Name()
names = GN.get_name(text=text)
print(names)

['越女', '芳心', '歐陽修', '柳永', '歐', '歌聲', '理宗', '晚風', '小妮子', '程', '陸', '無雙', '程英', '鬍鬚', '鬚髮', '陸無雙', '滋味', '程英與表妹', '何沅君', '人', '淚水', '老伯伯', '怪客', '頭上', '阿沅', '陸展元', '公公', '手', '何', '陸展元這小畜生', '聲音中', '大媽', '陸無', '程英和陸無雙手', '頭上熱氣', '少女', '陸門', '笑聲', '陸立鼎', '父親陸立鼎', '李莫愁', '赤練仙子', '赤練', '哥哥', '阿根', '男僕阿根', '陸二娘', '小英', '陸家滿門', '陸氏', '陸氏夫婦', '身子', '牆頭那男孩', '女孩兒家', '男孩', '程英與另一個男孩', '陸立鼎夫婦', '李', '對方', '婦人', '高', '食中兩指', '武', '武三通', '一燈', '我家官人', '武娘子', '一燈大師', '陸爺', '拙夫', '拙夫和李莫愁', '雙方', '江南人', '令兄', '武氏', '尊夫這等行逕', '陸爺陸二娘', '武敦儒', '武修文', '弟弟', '武學名家的兩個兒子', '郭靖', '黃蓉', '靖蓉', '蓮蓬', '修文', '敦儒', '父親', '啼聲', '聲音', '雙鵰', '爹爹', '鼻子', '芙兒', '武修文鼻血', '段智興', '天色', '陸展元一家', '武三爺', '夫婦', '膚色', '洪凌波', '武娘子與陸二娘', '柯鎮惡', '鐵杖', '黃藥師', '洪', '郭芙', '女兒', '雞', '黃', '靖蓉夫婦', '郭靖、黃蓉', '梅超風', '陸郎', '柯', '郭氏', '郭氏夫婦', '銀絲', '肩膀', '桃花島', '柯鎮惡與武娘子', '黃影', '劍頭', '金刃', '格格', '火勢']
